In [1]:
import cv2
import mediapipe as mp
import numpy as np
from collections import deque

In [2]:
mp_pose = mp.solutions.pose
mp_draw = mp.solutions.drawing_utils

In [9]:


pose = mp_pose.Pose(
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    model_complexity=1
)


def get_angle(a, b, c):
    a = np.array([a.x, a.y])
    b = np.array([b.x, b.y])
    c = np.array([c.x, c.y])

    ba = a - b
    bc = c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-10)
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

# =========================
# STANCE CLASSIFIER
# =========================
def classify_kungfu_pro(lm):
    torso_len = np.sqrt((lm[11].x - lm[23].x)**2 + (lm[11].y - lm[23].y)**2) + 1e-10

    l_knee = get_angle(lm[23], lm[25], lm[27])
    r_knee = get_angle(lm[24], lm[26], lm[28])

    feet_dist = abs(lm[27].x - lm[28].x) / torso_len
    ankle_diff = abs(lm[27].y - lm[28].y) / torso_len

    is_crossed = (lm[27].x > lm[28].x and lm[11].x < lm[12].x) or \
                 (lm[28].x > lm[27].x and lm[12].x < lm[11].x)

    detected_pose = None
    color = (180, 180, 180)

   #conditions
   

    if feet_dist > 1.7 and (l_knee < 75 or r_knee < 75):
        detected_pose = "PU BU (Drop Stance)"
        color = (0, 255, 0)

    elif is_crossed and l_knee < 95 and r_knee < 95:
        detected_pose = "XIE BU (Resting)"
        color = (255, 128, 0)

    elif ankle_diff > 0.65:
        detected_pose = "DU LI BU (Golden Rooster)"
        color = (0, 128, 255)

    elif feet_dist > 1.25 and l_knee < 135 and r_knee < 135:
        detected_pose = "MA BU (Horse Stance)"
        color = (255, 255, 0)

    elif feet_dist > 1.3 and (
        (l_knee < 115 and r_knee > 155) or
        (r_knee < 115 and l_knee > 155)
    ):
        detected_pose = "GONG BU (Bow Stance)"
        color = (0, 255, 255)

    elif feet_dist < 1.1 and (
        (l_knee < 110 and r_knee > 140) or
        (r_knee < 110 and l_knee > 140)
    ):
        detected_pose = "XU BU (Empty Stance)"
        color = (255, 0, 255)

    elif feet_dist < 0.6 and l_knee < 155 and r_knee < 155:
        detected_pose = "DING BU (T-Stance)"
        color = (150, 255, 150)

    # =========================
    # ❌ UNKNOWN POSE
    # =========================
    if detected_pose is None:
        return "UNKNOWN POSE", (0, 0, 255)

    return detected_pose, color

# =========================
# CAMERA + BUFFER
# =========================
cap = cv2.VideoCapture(0)
buffer = deque(maxlen=12)

# =========================
# MAIN LOOP
# =========================
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    display_label = "WAITING..."
    display_color = (255, 255, 255)

    if results.pose_landmarks:
        mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        raw_label, raw_color = classify_kungfu_pro(results.pose_landmarks.landmark)
        buffer.append(raw_label)

        display_label = max(set(buffer), key=buffer.count)
        if display_label == raw_label:
            display_color = raw_color

    # =========================
    # UI PANEL
    # =========================
    cv2.rectangle(frame, (0, 0), (640, 95), (15, 15, 15), -1)
    cv2.putText(frame, "KUNG FU MASTER AI - 8 STANCES", (20, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (180,180,180), 1)

    cv2.putText(frame, display_label, (20, 80),
                cv2.FONT_HERSHEY_TRIPLEX, 1.2, display_color, 2)

    cv2.imshow("Kung Fu Master AI", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
